<a href="https://colab.research.google.com/github/WalterZStark/proprioceptive-robotic-hand/blob/main/Image_Detection_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Import Packages

import torch
import sklearn

import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt
#from sklearn.preprocessing import normalize


# Allows torch to use gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Get Data Transforms for ReseNet50
train_transforms = ResNet50_Weights.DEFAULT.transforms()

# # Define data transforms
# transform = transforms.Compose([
#       transforms.Resize((224,224)),
#       transforms.ToTensor()])


# Import images from Oxford Pet Dataset
oxford_train_data = datasets.OxfordIIITPet(root = './data-train', split = "trainval", transform = train_transforms, download = True)
oxford_test_data = datasets.OxfordIIITPet(root = './data-test', split = "test", transform = train_transforms, download = True)

# Implement Data Loader
batch = 64
workers = 4
data_train_loader = DataLoader(oxford_train_data,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=workers)

data_test_loader = DataLoader(oxford_test_data,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=workers)





100%|██████████| 791918971/791918971 [00:31<00:00, 25145901.68it/s]


Extracting data-train/oxford-iiit-pet/images.tar.gz to data-train/oxford-iiit-pet


100%|██████████| 19173078/19173078 [00:01<00:00, 9988756.15it/s] 


Extracting data-train/oxford-iiit-pet/annotations.tar.gz to data-train/oxford-iiit-pet


100%|██████████| 791918971/791918971 [00:40<00:00, 19771960.40it/s]


Extracting data-test/oxford-iiit-pet/images.tar.gz to data-test/oxford-iiit-pet


100%|██████████| 19173078/19173078 [00:01<00:00, 10026389.25it/s]


Extracting data-test/oxford-iiit-pet/annotations.tar.gz to data-test/oxford-iiit-pet


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [28]:


# Generate embeddings

def generate_embeddings(oxford_train_data):
    """
    Transform, resize and normalize the images and then use a pretrained model to extract
    the embeddings.
    """




    # TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
    #  more info here: https://pytorch.org/vision/stable/models.html)



    embeddings = []
    embedding_size = 2048 #CHANGED FROM 4096

    # pick your model
    num_images = len(oxford_train_data)

    embeddings = np.zeros((num_images, embedding_size))

    # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
    # model to access the embeddings the model generates.



    model = resnet50(pretrained = True)
    model.to(device)
    model = nn.Sequential(*(list(model.children())[:-1]))
    i = 0
    ran = False
    for [X, y] in oxford_train_data:
        '''
        if not ran:

            # Display Images
            convert_tensor = transforms.Grayscale()
            img_g = convert_tensor(X[0,:3,:,:])
            print('Grayscale Image Dimensions', img_g.shape)

            # display grayscale image
            plt.imshow(img_g.permute(1, 2, 0), cmap='gray')
            plt.show()



            output = model(X)
            print(output)
            print(output.shape)
            ran = True
        '''

        X = X.unsqueeze(0)
        output = model(X)
        #print(len(output.detach().numpy().flatten()))
        #embeddings[i] = output.detach().numpy().flatten()

        for j in range(0,batch):
            if j >= len(output) or batch*i+j > num_images:
                break
            embeddings[batch*i+j] = output[j].detach().numpy().squeeze()


        #print(len(embeddings[i]))

        i += 1


    return embeddings


# Training sequence
# Train Our Own Neural Network

    # Loss Function



# Testing sequence

# Output Results

if __name__ == '__main__':
  training_embeddings_set = generate_embeddings(oxford_train_data)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
